This code fetches all workspaces and their datasets from the Power BI admin API in batches, adds the workspace ID to each dataset, and creates Spark DataFrames for the workspaces and datasets.

In [ ]:
#########################################################################################
# Install modules etc.
#########################################################################################

import requests
import json

#########################################################################################
# Read secrets from Azure Key Vault
#########################################################################################
key_vault = ""
tenant_id = mssparkutils.credentials.getSecret(key_vault , "FabricTenantId")
client_id = mssparkutils.credentials.getSecret(key_vault , "AdminApiClientId")
client_secret = mssparkutils.credentials.getSecret(key_vault , "AdminApiClientSecret")

#########################################################################################
# Authentication - Replace string variables with your relevant values       
#########################################################################################

try:
    from azure.identity import ClientSecretCredential
except Exception:
    !pip install azure.identity
    from azure.identity import ClientSecretCredential
from pyspark.sql import SparkSession

#########################################################################################
#Fetch all workspaces in premium capacities
#########################################################################################
# Define the power bi admin api endpoint

powerbi_url = "https://api.powerbi.com/v1.0/myorg/admin/groups"
api_gateway_uri = "https://api.powerbi.com"
# Define the scope for power bi service
scope = "https://analysis.windows.net/powerbi/api/.default"

# Define the oauth2 token endpoint
token_url = f"https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/token"

# Define the payload for requesting the access token
payload = {
    "grant_type": "client_credentials",
    "client_id": client_id,
    "client_secret": client_secret,
    "scope": scope
}


# Make a post request to get the access token
response = requests.post(token_url, data=payload)


# Check if the request was successful
if response.status_code == 200:
    # Get the access token from the response
    access_token = response.json()["access_token"]
    headers = {
        "Authorization": f"Bearer {access_token}"
    }

# Initialize empty lists for workspaces, datasets, dashboards, reports, users and dataflows
workspaces = []
datasets = []
dashboards = []
reports = []
users = []
dataflows = []

# Initialize batch and batch_size for pagination
batch = 0
batch_size = 1000

# Start a loop to fetch all workspaces in batches
while True:
    # Calculate the number of workspaces to skip based on the current batch
    skip = batch * batch_size

    # Make a GET request to fetch a batch of workspaces
    workspace_batch = requests.get("https://api.powerbi.com/v1.0/myorg/admin/groups?%24top={}&%24expand=reports%2C%20dashboards%2C%20datasets%2C%20users%2C%20dataflows&%24skip={}&%24filter=isOnDedicatedCapacity%20eq%20true".format(batch_size,skip), headers=headers)

    # Parse the response JSON
    workspace_data = workspace_batch.json()

    # Get the total number of workspaces
    total_workspaces = workspace_data.get("@odata.count", 0)

    # If the current batch of workspaces is not empty
    if len(workspace_data.get("value", [])) > 0:
        # Add the workspaces to the workspaces list
        workspaces.extend(workspace_data["value"])

        # For each workspace, extract its datasets and add them to the datasets list
        for workspace in workspace_data["value"]:
            if len(workspace.get("datasets", [])) > 0:
                for dataset in workspace["datasets"]:
                    # Add workspaceId to each dataset
                    dataset["workspaceId"] = workspace["id"]
                # Add the datasets to the datasets list
                datasets.extend(workspace["datasets"])
        
        # For each workspace, extract its reports and add them to the reports list
        for workspace in workspace_data["value"]:
            if len(workspace.get("reports", [])) > 0:
                for report in workspace["reports"]:
                    # Add workspaceId to each report
                    report["workspaceId"] = workspace["id"]
                # Add the reports to the reports list
                reports.extend(workspace["reports"])

        # Extract dashboards from workspace objects
        for workspace in workspace_data["value"]:
            if len(workspace.get("dashboards", [])) > 0:
                for dashboard in workspace["dashboards"]:
                    dashboard["workspaceId"] = workspace["id"]
                dashboards.extend(workspace["dashboards"])  

        # Extract users from workspace objects
        for workspace in workspace_data["value"]:
            if len(workspace.get("users", [])) > 0:
                for user in workspace["users"]:
                    user["workspaceId"] = workspace["id"]
                users.extend(workspace["users"])
        
        # Extract dataflows from workspace objects
        for workspace in workspace_data["value"]:
            if len(workspace.get("dataflows", [])) > 0:
                for dataflow in workspace["dataflows"]:
                    dataflow["workspaceId"] = workspace["id"]
                dataflows.extend(workspace["dataflows"])     


    # Move to the next batch
    batch += 1

    # If we have fetched all workspaces, break the loop
    if skip >= total_workspaces:
        break

#Create spark dataframes and define schema
df_workspaces = spark.createDataFrame(workspaces, schema="id string, isReadOnly string, dataflowStorageId string, description string, pipelineId string, isOnDedicatedCapacity string, capacityId string, capacityMigrationStatus string, defaultDatasetStorageFormat string, type string, state string, hasWorkspaceLevelSettings  string, name string")
df_datasets = spark.createDataFrame(datasets, schema="workspaceId string, id string, name string, addRowsAPIEnabled string,configuredBy string, isRefreshable string, isEffectiveIdentityRequired string, isEffectiveIdentityRolesRequired string, isOnPremGatewayRequired string, targetStorageMode string, contentProviderType string, createdDate string")
df_reports = spark.createDataFrame(reports, schema="workspaceId string, appId string, createdBy string, createdDateTime string, description string, embedUrl string, id string, modifiedBy string, modifiedDateTime string, reportType string, originalReportId string, webUrl string, name string, datasetId string")
df_dashboards = spark.createDataFrame(dashboards, schema="workspaceId string, id string, displayName string, isReadOnly string")
df_users = spark.createDataFrame(users,  schema="workspaceId string, emailAddress string, groupUserAccessRight string, identifier string, principalType string, displayName string")
df_dataflows = spark.createDataFrame(dataflows,  schema="workspaceId string, name string, description string, configuredBy string, modelUrl string")

# Print the total number of workspaces, reports, dashboards, and datasets for testing purposes
print("Total workspaces:", len(workspaces))
print("Total reports:", len(reports))
print("Total dashboards:", len(dashboards))
print("Total datasets:", len(datasets))
print("Total users:", len(users))
print("Total dataflows:", len(dataflows))

# Write to table
df_workspaces.write.mode("overwrite").format("delta").saveAsTable("Workspaces")
df_datasets.write.mode("overwrite").format("delta").saveAsTable("Datasets")
df_reports.write.mode("overwrite").format("delta").saveAsTable("Reports")
df_dashboards.write.mode("overwrite").format("delta").saveAsTable("Dashboards")
df_users.write.mode("overwrite").format("delta").saveAsTable("WorkspaceUsers")
df_dataflows.write.mode("overwrite").format("delta").saveAsTable("Dataflows")


